In [125]:
import math
import numpy as np
import pandas as pd
from scipy.stats import t, norm, ttest_1samp, ttest_ind, ttest_rel, f_oneway, chisquare
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings

In [111]:
class ZTest:
    """Perform a Z-test for a single sample"""
    def __init__(self, sample:list, std, alpha:float, pop_mean):
        """
        Parameters:
        -----------
        sample: list
            A list of numeric values representing the sample data
        std: float
            The population standard deviation
        alpha: float
            The significance level for the hypothesis test
        pop_mean: float
            The population mean to compare the sample mean against
        """
        assert isinstance(sample, list), "Sample must be a list of numeric values."
        assert std > 0, "Standard deviation must be positive."
        assert 0 < alpha < 1, "Alpha must be between 0 and 1."

        self.sample = sample
        self.std = std
        self.alpha = alpha
        self.pop_mean = pop_mean

    def test(self) -> str:
        """Perform the Z-test"""
        sample_mean = np.array(self.sample).mean()
        sample_size = len(self.sample)
        z = (sample_mean - self.pop_mean) / (self.std / math.sqrt(sample_size))
        p_value = 2 * (1 - norm.cdf(abs(z)))

        if p_value > self.alpha:
            print('Fail to reject H0 (Accept H0). The data are consistent with a hypothesized population mean equal to', self.pop_mean)
        else:
            print('Reject H0 (Accept H1). The data are not consistent with a hypothesized population mean equal to', self.pop_mean)

    def degree_of_freedom(self) -> int:
        """Calculate the degrees of freedom for the Z-test"""
        df = len(self.sample) - 1
        return df

    def critical_value(self) -> str:
        """Calculate the critical value for the Z-test"""
        df = self.degree_of_freedom()
        cv = t.ppf(1 - self.alpha / 2, df)  # critical value
        print(f"The critical value given the degree of freedom and the alpha = (+/-) {cv:.3f}")

1.	Z-test:
Suppose we are testing a new model of cell-phone and want to compare its bandwidth to the previous model.

Sample = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428,  4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]


•	State the null and alternative hypotheses.
•	Use the Z-test to determine whether the new model has a significantly different bandwidth 4.5 GHz from the previous model with a standard deviation of 0.6 GHz, using a significance level of 0.05.
•	Interpret the results


In [112]:
Sample = [4.186, 4.439, 4.781, 4.388, 4.947, 4.853, 4.889, 4.682, 4.428,  4.533, 4.557, 4.761, 4.491, 4.334, 4.83 , 4.268, 4.68 , 4.437, 5.382, 5.111, 5.096, 5.232, 5.033, 5.57 , 4.474, 4.789, 4.725, 4.84 , 4.817, 4.438, 4.754, 4.966, 4.285, 4.482, 4.396, 4.418, 4.514, 5.383, 5.264, 4.309, 5.058, 4.392, 4.788, 4.934, 4.967, 4.554, 4.42 , 5. , 5.126, 5.082, 4.944, 4.658]

std = 0.6
alpha = 0.05
pop_mean = 4.5
testing = ZTest(Sample, std, alpha, pop_mean)
testing.test()

Reject H0 (Accept H1). The data are not consistent with a hypothesized population mean equal to 4.5


In [113]:
class TTest:
    """Perform a t-test"""
    def __init__(self, pop_mean=0):
        """
        Parameters:
        -----------
        sample: list
            A list of numeric values representing the first sample data
        alpha: float
            The significance level for the hypothesis test
        std: float, optional
            The population standard deviation (only for one-sample t-test)
        pop_mean: float, optional
            The population mean to compare the sample mean against (only for one-sample t-test)
        """

        self.pop_mean = pop_mean
    
    def check(self, p_value, alpha) -> str:
        if p_value > alpha:
            print('Fail to reject H0 (Accept H0). The mean of the population is equal to', self.pop_mean)
        else:
            print('Reject H0 (Accept H1). The mean of the population is NOT equal to', self.pop_mean)

    def one_sample_test(self, sample:list, alpha:float) -> str:
        assert isinstance(sample, list), "Sample must be a list of numeric values."

        df = len(sample) - 1
        sample_mean = np.array(sample).mean()
        sample_size = len(sample)
        sample_std = np.std(np.array(sample), ddof = 1)
        
        t_statistic = (sample_mean - self.pop_mean) / (sample_std / math.sqrt(sample_size))
        p_value = 2 * (1 - t.cdf(abs(t_statistic), df))
        
        
        # t_stat, p_value = ttest_1samp(self.sample, self.pop_mean)

        print(f"t-statistic: {t_statistic:.2f}")
        print(f"p-value:: {p_value:.2f}")

        self.check(p_value, alpha)

    def two_sample_test(self, sample1, sample2, alpha:float) -> str:
        assert isinstance(sample2, list), "Sample 2 must be a list of numeric values."
        assert isinstance(sample1, list), "Sample 1 must be a list of numeric values."


        # t_statistic, p_value = ttest_ind(self.sample, sample2, equal_var=False)
        n1 = len(sample1)
        n2 = len(sample2)
        df = n1 + n2 - 2
        
        sample1_mean = np.array(sample1).mean()
        sample2_mean = np.array(sample2).mean()

        var1 = np.array(sample1).var(ddof=1)
        var2 = np.array(sample2).var(ddof=1)

            

        t_statistic = (sample1_mean - sample2_mean) / (math.sqrt(var1/n1 + var2/n2))
        p_value = round(2 * (1 - t.cdf(abs(t_statistic), df)),5)
        

        print(f"t-statistic: {t_statistic:.2f}")
        print(f"p-value: {p_value:.2f}")

        self.check(p_value, alpha)


    def paired_t_test(self, before:list, after:list, alpha:float) -> str:
        assert isinstance(before, list), "before must be a list of numeric values."
        assert isinstance(after, list), "after must be a list of numeric values."

        # t_statistic, p_value = ttest_rel(self.sample, sample2)

        n = len(before)
        df = n - 1
        
        diff = np.array(after) - np.array(before)
        std = diff.std(ddof=1)
        
        self.one_sample_test(diff.tolist(), 0)

        t_statistic = diff.mean() / (std / math.sqrt(n))
        p_value = 2 * (1 - t.cdf(abs(t_statistic), df))
        
        print(f"t-statistic: {t_statistic:.2f}")
        print(f"p-value: {p_value:.2f}")

        self.check(p_value, alpha)

One sample T-test: Suppose we have a new manufacturing process for producing aluminum cans, and we want to test whether the mean weight of the cans produced using this new process is significantly different from the target weight of 15 grams. We randomly sample 30 cans produced using the new process and measure their weights in grams. We obtain the following data: 

Sample = [14.8, 15.2, 15.1, 15.3, 15.0, 14.9, 15.2, 14.8, 15.1, 15.0, 14.9, 14.8, 15.2, 14.9, 15.0, 14.9, 15.1, 15.3, 15.0, 15.1, 14.8, 15.0, 15.2, 15.1, 15.3, 15.1, 15.0, 14.8, 15.2, 15.0]

•	State the null and alternative hypotheses.
•	Use the appropriate T-test to determine whether the mean weight of the produced cans is equal to the target weight of 15 grams, using a significance level of 0.05.
•	Interpret the results

In [114]:
Sample = [14.8, 15.2, 15.1, 15.3, 15.0, 14.9, 15.2, 14.8, 15.1, 15.0, 14.9, 14.8, 15.2, 14.9, 15.0, 14.9, 15.1, 15.3, 15.0, 15.1, 14.8, 15.0, 15.2, 15.1, 15.3, 15.1, 15.0, 14.8, 15.2, 15.0]

alpha=0.05
pop_mean=15
    
TTest(pop_mean).one_sample_test(Sample, alpha)

t-statistic: 1.27
p-value:: 0.22
Fail to reject H0 (Accept H0). The mean of the population is equal to 15


Two Sample T-test: Suppose a food company has developed a new flavor of potato chips and wants to compare it to the current best-selling flavor. The company randomly selects two groups of 20 customers each. The first group is given the new flavor of potato chips, while the second group is given the best-selling flavor. After trying the potato chips, each customer rates the flavor on a scale of 1 to 10. The following are the flavor ratings for the two groups:

New flavor = [8, 7, 9, 6, 7, 8, 9, 7, 8, 7, 6, 8, 7, 9, 8, 7, 6, 9, 8, 7]
Best selling flavor = [6, 7, 8, 6, 7, 6, 7, 6, 8, 7, 6, 7, 6, 8, 7, 6, 7, 8, 6, 7]

State the null and alternative hypotheses.
Use the appropriate T-test to determine whether there is a significant difference between the two groups or not, using a significance level of 0.05.
Interpret the results

In [115]:
New_flavor = [8, 7, 9, 6, 7, 8, 9, 7, 8, 7, 6, 8, 7, 9, 8, 7, 6, 9, 8, 7]
Old_flavor = [6, 7, 8, 6, 7, 6, 7, 6, 8, 7, 6, 7, 6, 8, 7, 6, 7, 8, 6, 7]
alpha = 0.05

TTest().two_sample_test(Old_flavor,New_flavor, alpha)

t-statistic: -2.66
p-value: 0.01
Reject H0 (Accept H1). The mean of the population is NOT equal to 0


Paired T-test: Suppose a company wants to evaluate a new training program for its employees. The company selects 20 employees and measures their productivity before and after the training program. The following are the productivity scores (number of tasks completed per hour) for each employee before and after the training:

Before = [15, 18, 12, 10, 17, 16, 12, 14, 19, 18, 11, 13, 16, 17, 19, 14, 16, 13, 15, 12]
After = [18, 20, 15, 13, 19, 18, 14, 16, 21, 20, 14, 16, 19, 20, 22, 16, 18, 15, 17, 14]

State the null and alternative hypotheses.
Use the appropriate T-test to determine whether the new training program has had an effect on employee productivity, with a significance level of 0.05.


In [116]:
Before = [15, 18, 12, 10, 17, 16, 12, 14, 19, 18, 11, 13, 16, 17, 19, 14, 16, 13, 15, 12]
After = [18, 20, 15, 13, 19, 18, 14, 16, 21, 20, 14, 16, 19, 20, 22, 16, 18, 15, 17, 14]
alpha = 0.05
TTest(alpha).paired_t_test(Before, After, alpha)

t-statistic: 20.91
p-value:: 0.00
Fail to reject H0 (Accept H0). The mean of the population is equal to 0.05
t-statistic: 21.35
p-value: 0.00
Reject H0 (Accept H1). The mean of the population is NOT equal to 0.05


In [117]:
class ANOVA_Test():
    def __init__(self, alpha = 0.05):
        self.alpha = alpha
    
    def one_way_anova(self, sample1:list, sample2:list, sample3:list, alpha:float) -> str:
        all_samples = [sample1, sample2, sample3]

        f_statistic, p_value = f_oneway(*all_samples)

        if p_value < self.alpha:
            print('Reject the null hypothesis H0, There is one group mean different from the others')
        else:
            print('Fail to reject the null hypothesis, all the group means are equal')

    def two_way_anova(self, data, dep_var, factor_vars):
        # Fit the two-way ANOVA model
        model_str = f"{dep_var} ~ {' + '.join(factor_vars)} + {'*'.join(factor_vars)}"
        model = ols(model_str, data=data).fit()
        table = sm.stats.anova_lm(model, typ=2)
        
        # Print the ANOVA table
        print(table)

One-Way: Suppose a company has three departments (A, B, and C) and wants to test whether there is a significant difference in salaries between the departments. The company selects 10 employees randomly from each department and records their salaries.

Department A = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59]
Department B = [50, 52, 48, 49, 55, 53, 51, 54, 47, 50]
Department C = [45, 43, 48, 50, 42, 47, 49, 46, 44, 48]

State the null and alternative hypotheses.
Use One-Way ANOVA-test to determine whether there is a significant difference in salaries between the three departments.
Interpret the results

In [118]:
alpha = 0.05
Department_A = [55, 60, 50, 58, 63, 62, 57, 56, 61, 59]
Department_B = [50, 52, 48, 49, 55, 53, 51, 54, 47, 50]
Department_C = [45, 43, 48, 50, 42, 47, 49, 46, 44, 48]

ANOVA_Test().one_way_anova(Department_A, Department_B, Department_C, alpha)

Reject the null hypothesis H0, There is one group mean different from the others


In [119]:
# Create a pandas DataFrame with the given data
data = pd.DataFrame({
    'department': ['A']*10 + ['B']*10 + ['C']*10,
    'gender': ['Male']*5 + ['Female']*5 + ['Male']*5 + ['Female']*5 + ['Male']*5 + ['Female']*5,
    'salary': [55, 60, 50, 58, 63, 62, 57, 56, 61, 59, 50, 52, 48, 49, 55, 53, 51, 54, 47, 50, 45, 43, 48, 50, 42, 47, 49, 46, 44, 48]
})

ANOVA_Test().two_way_anova(data=data, dep_var='salary', factor_vars=['department', 'gender'])

                       sum_sq    df          F        PR(>F)
department         718.466667   2.0  35.104235  7.472321e-08
gender               8.533333   1.0   0.833876  3.702368e-01
department:gender    3.266667   2.0   0.159609  8.533743e-01
Residual           245.600000  24.0        NaN           NaN


In [126]:
class ChiSquare:
    def __init__(self, alpha=0.05):
        self.alpha = alpha

    def test_goodness_of_fit(self, observed, expected):
        # Calculate the chi-square statistic and p-value
        chi_sq_statistic, p_value = chisquare(observed, expected)

        # Determine the degrees of freedom
        df = len(observed) - 1

        # Print the results
        print(f"Chi-square statistic: {chi_sq_statistic:.2f}")
        print(f"Degrees of freedom: {df}")
        print(f"p-value: {p_value:.4f}")

        # Determine the conclusion
        if p_value < self.alpha:
            print("Conclusion: Reject the null hypothesis at alpha =", self.alpha)
        else:
            print("Conclusion: Fail to reject the null hypothesis at alpha =", self.alpha)


In [130]:
# Define the observed and expected frequencies
observed = [18, 20, 16, 22, 14, 10]
N = 100 # simulation
expected = [N/len(observed)]*len(observed)

# Instantiate the ChiSquare class
chi_square = ChiSquare()

# Perform the chi-square test
chi_square.test_goodness_of_fit(observed, expected)

Chi-square statistic: 5.60
Degrees of freedom: 5
p-value: 0.3471
Conclusion: Fail to reject the null hypothesis at alpha = 0.05


In [131]:
class ABTesting(TTest):
    """
    Perform an A/B test using the two-sample t-test
    """

    def __init__(self, sample1, sample2, alpha=0.05):
        """
        Parameters:
        -----------
        sample1: list
            A list of numeric values representing the first sample data
        sample2: list
            A list of numeric values representing the second sample data
        alpha: float, optional
            The significance level for the hypothesis test
        """

        super().__init__()

        self.sample1 = sample1
        self.sample2 = sample2
        self.alpha = alpha

    def run_test(self):
        """
        Run the two-sample t-test and print the results
        """

        self.two_sample_test(self.sample1, self.sample2, self.alpha)

A/B testing: Suppose a company has launched a new flavor of soda and wants to test if it has a higher preference than the old flavor. The company conducts a survey with a sample of 30 customers, randomly split into two groups of 15. One group is given the old flavor of soda and the other group is given the new flavor of soda. Each participant rates their preference on a scale of 1 to 10.

Old Flavor = [6, 7, 8, 5, 6, 7, 5, 8, 6, 7, 5, 6, 7, 6, 5]
New Flavor = [8, 9, 7, 8, 9, 6, 7, 8, 7, 8, 7, 8, 9, 6, 8]

State the null and alternative hypotheses.
Use two sample t-test to determine whether there is a significant difference in preference between the old and new flavors of soda, with a significance level of 0.05
Interpret the results

In [134]:
Old_Flavor = [6, 7, 8, 5, 6, 7, 5, 8, 6, 7, 5, 6, 7, 6, 5]
New_Flavor = [8, 9, 7, 8, 9, 6, 7, 8, 7, 8, 7, 8, 9, 6, 8]

ABTesting(Old_Flavor,New_Flavor).run_test()

t-statistic: -3.82
p-value: 0.00
Reject H0 (Accept H1). The mean of the population is NOT equal to 0
